# Imports 
Necessary imports, based in part on [this](https://www.kaggle.com/code/awsaf49/planttraits2024-kerascv-starter-notebook)

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow" # you can also use tensorflow or torch

# deep learning
import keras_cv
import keras
from keras import ops
import tensorflow as tf
import cv2
import umap.umap_ as umap

# data processing
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
import joblib
import sklearn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pickle

# stats
from sklearn.metrics import mutual_info_score
from scipy.stats import wasserstein_distance

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
from keras.utils import plot_model
import pydot

# misc 
from config import config

# Data pre-processing

## Loading
The general format of each instance of the data is 
```
[ids*, ancillary data*, trait means*, traits sd*, image path]
```

In [ ]:
from config import config
DATA_PATH = "../data"

# creating df that has image path and related ancillary data
df = pd.read_csv(DATA_PATH + "/train.csv")
df['image_path'] = f'{DATA_PATH}/train_images/' + df['id'].astype(str) + '.jpeg'
df.loc[:, config.aux_class_names] = df.loc[:, config.aux_class_names].fillna(-1)
display(df.head(2))
print(df.shape[0]) 

# same df but for the test data
test_df = pd.read_csv(DATA_PATH + "/test.csv")
test_df['image_path'] = f'{DATA_PATH}/test_images/'+ test_df['id'].astype(str) + '.jpeg'
FEATURE_COLS = list(test_df.columns[1:-1])
display(test_df.head(2))
print(test_df.shape[0])

## traits

### mean traits

In [ ]:
mean_traits = df.iloc[:, 164:170]
sd_traits = df.iloc[:, 170:176]
display(mean_traits.head(2))


def mean_normalizer(col):
    mean = mean_traits.iloc[:, col]
    mean = mean.drop_duplicates()
    return (mean - mean.min()) / (mean.max() - mean.min())

plt.figure(figsize=(10, 5))
for i in range(6): 
    plt.hist(mean_normalizer(i), log=True, bins=100, alpha=0.5, label=f'Normalized x{i} mean')
    
plt.title('Normalized mean trait distributions')
plt.xlabel('Normalized mean trait value')
plt.ylabel('Log count')
plt.legend()
plt.show()

In [ ]:
x_11_mean = mean_traits.iloc[:, 1]
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# remove outliers
x_11_mean = x_11_mean[x_11_mean.between(x_11_mean.quantile(0), x_11_mean.quantile(1))]

x_11_mean = x_11_mean.drop_duplicates()

ax[0].hist(x_11_mean, bins=100, log=True, alpha=0.5)
ax[0].set_title('x11 mean distribution')
ax[0].set_xlabel('x11 mean value')
ax[0].set_ylabel('Log count')

x_11_mean = x_11_mean[x_11_mean.between(x_11_mean.quantile(0), x_11_mean.quantile(0.99))]
print(x_11_mean.count())
ax[1].hist(x_11_mean, bins=100, alpha=0.5)
ax[1].set_title('x11 mean distribution (99th percentile)')
ax[1].set_xlabel('x11 mean value')
ax[1].set_ylabel('Instance count')

In [ ]:
fig, ax = plt.subplots(1, 6, figsize=(30, 5))

for i in range(6):
    x_mean = mean_traits.iloc[:, i]
    x_mean = x_mean[x_mean.between(x_mean.quantile(0), x_mean.quantile(0.99))]
    x_mean = x_mean.drop_duplicates()
    
    ax[i].hist(x_mean, bins=100, log=True, alpha=0.5, label=f'Normalized x{i} mean')
    ax[i].set_title(f'x{i} mean distribution (99th percentile)')
    ax[i].set_xlabel(f'x{i} mean value')
    ax[i].set_ylabel('Log count')

This gives us some idea of the distribution of the targets we are training for. One thing to consider is that this distribution of traits might not be representative of the general traits of plant species. 
*Notes*
- we can maybe check some location data to see where most of the images came from to see if certain traits indicate a geographic bias 

### sd traits

In [ ]:
sd_traits = df.iloc[:, 170:176]
display(sd_traits.head(2))
def sd_normalizer(col):
    sd = sd_traits.iloc[:, col]
    return (sd - sd.min()) / (sd.max() - sd.min())

plt.figure(figsize=(10, 5))
for i in range(6):  
    plt.hist(sd_normalizer(i), log=True, bins=100, alpha=0.5, label=f'Normalized SD x{i}')

plt.title('Normalized standard deviation trait distributions')
plt.legend()
plt.show()


This might be an even better representative that we are training towards a biased sample. Intuitively I'd assume that traits would probably have a large variance if we have a geographic diversity in our sample, but here it seems this is less so the case for most of the deviations maybe ?

## ancillary data

In [ ]:
ancillary_data = df.iloc[:, 1:164]

fig = plt.figure(figsize=(10, 10))

# worldclim 
worldclim = ancillary_data.filter(like='WORLDCLIM')

g = sns.pairplot(worldclim, diag_kind='kde', plot_kws={'alpha': 0.5})

# remove labels 
for ax in g.axes.flatten():
    ax.set_ylabel('')
    ax.set_xlabel('')

In [ ]:
worldclim = ancillary_data.filter(like='WORLDCLIM')

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
corr = worldclim.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', cbar=True)
ax.set_title('Worldclim feature correlation heatmap')

high_corr = [i for i in corr[corr > 0.9].stack().index.tolist() if i[0] != i[1]]
high_corr = list(set([tuple(sorted(i)) for i in high_corr]))

scaled_worldclim_data = (worldclim - worldclim.min()) / (worldclim.max() - worldclim.min())
print(high_corr)

# PCA
pca = PCA(n_components=0.95)
pca_result = pca.fit_transform(scaled_worldclim_data)

fig = plt.figure(figsize=(20, 10))

# Plot PCA
ax1 = fig.add_subplot(121, projection='3d')
ax1.scatter(pca_result[:, 0], pca_result[:, 1], pca_result[:, 2], c=pca_result[:, 0], cmap='viridis', s=10)
ax1.set_title('PCA of worldclim features')

reconstructed_pca = pca.inverse_transform(pca_result)
loss_pca = np.mean((scaled_worldclim_data - reconstructed_pca) ** 2)

print(f'PCA Reconstruction Loss: {loss_pca}')

# UMAP
reducer = umap.UMAP(n_components=3)
embedding = reducer.fit_transform(scaled_worldclim_data)

# Plot UMAP
ax2 = fig.add_subplot(122, projection='3d')
ax2.scatter(embedding[:, 0], embedding[:, 1], embedding[:, 2], c=embedding[:, 0], cmap='viridis', s=10)
ax2.set_title('UMAP embedding of worldclim features')

reconstructed_umap = reducer.inverse_transform(embedding)
loss_umap = np.mean((scaled_worldclim_data - reconstructed_umap) ** 2)

print(f'UMAP Reconstruction Loss: {loss_umap}')

plt.show()

So we start of by just looking at worldclim subset of our training data and testing out some of the basic methods we are going to then use on the broader dataset, namely finding correlations and then comparing the performance of PCA vs UAMP in the dimensionality reduction.

In [ ]:
ancillary_data = df.iloc[:, 1:164]
 
worldclim   = ancillary_data.filter(like='WORLDCLIM')
soil        = ancillary_data.filter(like='SOIL')
modis       = ancillary_data.filter(like='MODIS')
vod         = ancillary_data.filter(like='VOD')

print(
    f"lengths {len(worldclim.columns)}, {len(soil.columns)}, {len(modis.columns)}, {len(vod.columns)}"
)
    

In [ ]:
# 3 x 1 subplot of correlation heatmaps 
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# soil 

soil_corr = soil.corr()
sns.heatmap(soil_corr, ax=ax[0], cmap='coolwarm', annot=False)

ax[0].set_title('Soil feature correlation heatmap')
ax[0].set_xticklabels([])
ax[0].set_yticklabels([])
ax[0].set_xlabel("Feature index")
ax[0].set_ylabel("Feature index")

# modis

modis_corr = modis.corr()
sns.heatmap(modis_corr, ax=ax[1], cmap='coolwarm', annot=False)

ax[1].set_title('MODIS feature correlation heatmap')
ax[1].set_xticklabels([])
ax[1].set_yticklabels([])
ax[1].set_xlabel("Feature index")
ax[1].set_ylabel("Feature index")

# vod

vod_corr = vod.corr()
sns.heatmap(vod_corr, ax=ax[2], cmap='coolwarm', annot=False)

ax[2].set_title('VOD feature correlation heatmap')
ax[2].set_xticklabels([])
ax[2].set_yticklabels([])
ax[2].set_xlabel("Feature index")
ax[2].set_ylabel("Feature index")

fig.tight_layout()
plt.show()

In [ ]:
correlations = [worldclim.corr(), soil.corr(), modis.corr(), vod.corr()]

high_corr = [] 

for corr in correlations:
    high_corr += [
        i for i in corr[corr > 0.9].stack().index.tolist() if i[0] != i[1]
    ]
    
mod_corr = []

for corr in correlations:
    mod_corr += [
        i for i in corr[corr > 0.65].stack().index.tolist() if i[0] != i[1]
    ]
    
total_cols = 164 
all_comb_num = total_cols * (total_cols - 1) / 2
    
high_corr = list(set([tuple(sorted(i)) for i in high_corr]))
print(
    f'num combinations with high correlation: {len(high_corr)}' + 
    f' ({len(high_corr) / all_comb_num * 100:.2f}%)' +
    f' tot = {all_comb_num}' 
)
print(
    f'num combinations with moderate correlation: {len(mod_corr)}' +
    f' ({len(mod_corr) / all_comb_num * 100:.2f}%)' +
    f' tot = {all_comb_num}'
)

In [ ]:
from config import config

scaler = StandardScaler()
ancillary_data = df.iloc[:, 1:164]

# random sample of ancillary data subset 
ancillary_data = scaler.fit_transform(ancillary_data)

# PCA 
pca = PCA(n_components=0.95)
pca_result = pca.fit_transform(ancillary_data)

# UMAP
reducer = umap.UMAP(n_components=30)
embedding = reducer.fit_transform(ancillary_data)

## missing data

In [ ]:
# iter over all rows in df and look for missing data

bad_rows = []
incomplete_cols = [n for n in config.aux_class_names if n.startswith("X") and n.endswith("_sd")]  # there is some missing data in the X*_sd columns
for i, row in df.iterrows():
    # check if any of those is a 0, -1 or 1
    if row[incomplete_cols].isin([0, -1, 1]).any():
        bad_rows.append(i)
        # replace them with nans instead
        df.loc[i, incomplete_cols] = np.nan

# remove duplicates
bad_rows = set(bad_rows)
percentage_bad = len(bad_rows) / len(df) * 100
print(f"Found {len(bad_rows)} bad rows ({len(bad_rows)}/{len(df)}) ({percentage_bad:.2f}%)")

# display top 5 bad rows
missing_data_df = df.iloc[list(bad_rows)]
missing_data_df.tail(5)

### where ?

Features marked as `X*_sd` are missing for almost 30% of the data (16387/55489) (29.53%). The rest of the data seems to be complete, with no other missing/zeroed values. We can ignore these rows, or we can simply ignore those features as it is a significant amount of data.

## data augmentation

### imputation of missing values

In [ ]:
# use KNN to try to fill the X*_sd columns
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5, missing_values=-1.0, weights='distance')

tabular = df.iloc[:, 170:176]

imputed_df = None

# if data not in csv already 
if not os.path.exists(f"{DATA_PATH}/train_knn_imputed.csv"):
    imputed_tabluar = imputer.fit_transform(tabular)
    imputed_df = pd.DataFrame(imputed_tabluar)
else: 
    imputed_df = pd.read_csv(f"{DATA_PATH}/train_knn_imputed.csv")
    

In [ ]:
display(tabular.head(5))

df = pd.concat([df.iloc[:, :170], tabular, df['image_path']], axis=1)
display(df.head(5))

In [ ]:
imputed_df = imputed_df.rename(columns={i: f"{config.aux_class_names[i]}" for i in range(6)})

# save the imputed data to a new csv in 'data/test_knn_imputed.csv'
if not os.path.exists(f"{DATA_PATH}/train_knn_imputed.csv"):
    df = pd.concat([df.iloc[:, :170], imputed_df, df['image_path']], axis=1)
    df.to_csv(DATA_PATH + '/train_knn_imputed.csv', index=False)

df = imputed_df.copy()

train_df = df.copy()

display(df.head(5))

### images

In [ ]:
def test_image_viewer(df):
    # slice the df to only include the columns we want
    img_df = df[["image_path"] + config.class_names]
# sample 5 rows 
    img_df = img_df.sample(5)

    img, traits = list(img_df["image_path"].values), list(img_df[config.class_names].values)

    num_imgs, num_cols = 5, 5

    plt.figure(figsize=(5 * num_cols, num_imgs // num_cols * 5))
    for i, (img, traits) in enumerate(zip(img, traits)):
        plt.subplot(num_imgs // num_cols, num_cols, i + 1)
        img = cv2.imread(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
        formatted_tar = "\n".join(
        [
            ", ".join(
                f"{name.replace('_mean','')}: {val:.2f}"
                for name, val in zip(config.class_names[j : j + 3], traits[j : j + 3])
            )
            for j in range(0, len(config.class_names), 3)
        ]
    )
    
        plt.imshow(img)
        plt.title(f"[{formatted_tar}]")
        plt.axis("off")
        
    plt.show()

test_image_viewer(df)

## data split

In [ ]:
from config import config
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=config.num_folds, shuffle=True, random_state=42)

# Create separate bin for each traits
for i, trait in enumerate(config.class_names):
    bin_edges = np.percentile(df[trait], np.linspace(0, 100, config.num_folds + 1))
    df[f"bin_{i}"] = np.digitize(df[trait], bin_edges)

df["final_bin"] = (
    df[[f"bin_{i}" for i in range(len(config.class_names))]]
    .astype(str)
    .agg("".join, axis=1)
)

# Perform the stratified split using final bin
df = df.reset_index(drop=True)
for fold, (train_idx, valid_idx) in enumerate(skf.split(df, df["final_bin"])):
    df.loc[valid_idx, "fold"] = fold

In [ ]:
fig, ax = plt.subplots(1, 6, figsize=(30, 5))

display(df.head(2))

avg_hist = {i: [] for i in range(6)}

for j in range(5):
    for i in range(6):
        fold_n = df[df["fold"] == j]
        plt_df = fold_n[config.class_names[i]]
        
        plt_df = plt_df[plt_df.between(plt_df.quantile(0), plt_df.quantile(0.99))]
        
        avg_hist[i].append(plt_df)
        plt_df = (plt_df - plt_df.min()) / (plt_df.max() - plt_df.min())
        
        ax[i].hist(plt_df, bins=100, log=True, alpha=0.5, label=f"Fold {j} {config.class_names[i]}")
        ax[i].set_title(config.class_names[i])
        ax[i].set_xlabel(f"{config.class_names[i]} value")
        ax[i].set_ylabel("Log count")
        ax[i].legend()
    
    
fig.tight_layout()
plt.show()

Cool ok so we know that visually the traits are equally distributed in the different folds, so that confirms that, now it might make sense to like quantifably check this. We can I think just average the histograms then compare the distance (I think its called?) with that of the original distribution.

In [ ]:
import pprint
print(list(avg_hist.values())[0][1].shape)
x4 = list(avg_hist.values())[0][1]
# print nan in x4
series_list = list(avg_hist.values())

series_list = list(map(lambda x: pd.DataFrame(x).sum(axis=0) / 5, series_list))

for i in range(6):
    print(series_list[i].isna().sum())

In [ ]:
mean_traits = df.iloc[:, 164:170]

fig, ax = plt.subplots(1, 6, figsize=(30, 5))

for i, (x_mean_col, avrg_mean) in enumerate(zip(mean_traits.columns, series_list)):
    x_mean = mean_traits[x_mean_col]
    x_mean = x_mean[x_mean.between(x_mean.quantile(0), x_mean.quantile(0.99))]
    x_mean = (x_mean - x_mean.min()) / (x_mean.max() - x_mean.min())

    avrg_mean = avrg_mean[:54934]

    # TODO - maybe use something better than wasserstein distance to eval the distributions
    print(f"Distribution difference for {config.class_names[i]}: {wasserstein_distance(x_mean, avrg_mean)}")

    avrg_mean = (avrg_mean - avrg_mean.min()) / (avrg_mean.max() - avrg_mean.min())
    ax[i].hist(x_mean, bins=100, log=True, alpha=0.5, label=f"Normalized {config.class_names[i]}")
    ax[i].hist(avrg_mean, bins=100, log=True, alpha=0.4, label=f"Averaged {config.class_names[i]} (5 folds)")
    ax[i].set_title(f"{config.class_names[i]} distribution ", fontsize=17)
    ax[i].set_xlabel(f"{config.class_names[i]} value", fontsize=17)
    ax[i].set_ylabel("Log count", fontsize=17)
    ax[i].legend()

fig.tight_layout()
plt.show()

In [ ]:
df = pd.read_csv(DATA_PATH + '/train_cleaned.csv')

In [ ]:
from sklearn.preprocessing import StandardScaler
from databuilder import build_dataset

# Sample from full data
sample_df = df.copy()
train_df = sample_df[sample_df.fold != config.fold]
valid_df = sample_df[sample_df.fold == config.fold]
print(f"# Num Train: {len(train_df)} | Num Valid: {len(valid_df)}")

# Normalize features
scaler = StandardScaler()
train_features = scaler.fit_transform(train_df[FEATURE_COLS].values)
valid_features = scaler.transform(valid_df[FEATURE_COLS].values)

# Train | img + geo
train_paths = train_df.image_path.values
train_labels = train_df[config.class_names].values
train_aux_labels = train_df[config.aux_class_names].values
train_ds_ig = build_dataset(train_paths, train_features, train_labels, train_aux_labels,
                         batch_size=config.batch_size, cache_dir=f"{DATA_PATH}/cached_training",
                         repeat=True, shuffle=True, augment=True, cache=False)

# Valid | img + geo
valid_paths = valid_df.image_path.values
valid_labels = valid_df[config.class_names].values
valid_aux_labels = valid_df[config.aux_class_names].values
valid_ds = build_dataset(valid_paths, valid_features, valid_labels, valid_aux_labels,
                         batch_size=config.batch_size, cache_dir=f"{DATA_PATH}/cached_training",
                         repeat=False, shuffle=False, augment=False, cache=False)

# Train | img
train_ds_i = build_dataset(train_paths, None,train_labels, train_aux_labels,
                         batch_size=config.batch_size, cache_dir=f"{DATA_PATH}/cached_training",
                         repeat=True, shuffle=True, augment=True, cache=False)

# Valid | img
valid_ds_i = build_dataset(valid_paths, None, valid_labels, valid_aux_labels,
                         batch_size=config.batch_size, cache_dir=f"{DATA_PATH}/cached_training",
                         repeat=False, shuffle=False, augment=False, cache=False)


In [ ]:
display(train_df.head(2))

# Model design

## loss function

In [ ]:
# loss function for model
from loss import R2Loss, R2Metric, adjusted_R2Loss

# testing r2
y_true = np.array([1, 2, 3, 4, 5])
y_pred = np.array([1, 1, 3, 4, 5])

r2 = R2Metric()
r2.update_state(y_true, y_pred)
print(r2.result())

## image model

### backbone comparison

In [ ]:
%load_ext autoreload
%autoreload 2

from loss import *
from config import config
from model import Model 

# comparing differen backbones
nn1 = Model(config, "efficientnetv2_b2_imagenet", True, "nn1")
nn2 = Model(config, "resnet50_imagenet", True, "nn2")
nn3 = Model(config, "efficientnetv2_s_imagenet", True, "nn3")

In [ ]:
from loss import *
from config import config
from model import Model 

nn1.summary()

In [ ]:
# 1 x 3 keras model plot 
fig, ax = plt.subplots(1, 3, figsize=(30, 10))

# TODO - causes errors
plot_model(nn1, to_file='model_nn1.png', show_shapes=True, show_layer_names=True)
plot_model(nn2, to_file='model_nn2.png', show_shapes=True, show_layer_names=True)
plot_model(nn3, to_file='model_nn3.png', show_shapes=True, show_layer_names=True)

# Load and display the images
img1 = plt.imread('model_nn1.png')
img2 = plt.imread('model_nn2.png')
img3 = plt.imread('model_nn3.png')

# Delete the images after loading them
os.remove('model_nn1.png')
os.remove('model_nn2.png')
os.remove('model_nn3.png')

ax[0].imshow(img1)
ax[0].axis('off')
ax[0].set_title('Model 1')

ax[1].imshow(img2)
ax[1].axis('off')
ax[1].set_title('Model 2')

ax[2].imshow(img3)
ax[2].axis('off')
ax[2].set_title('Model 3')

plt.show()

### training

In [ ]:
import math

def get_lr_callback(batch_size=8, mode='cos', epochs=10, plot=False):
    lr_start, lr_max, lr_min = 5e-5, 8e-6 * batch_size, 1e-5
    lr_ramp_ep, lr_sus_ep, lr_decay = 3, 0, 0.75

    def lrfn(epoch):  # Learning rate update function
        if epoch < lr_ramp_ep: lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep: lr = lr_max
        elif mode == 'exp': lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
        elif mode == 'step': 
            lr = lr_max * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep) // 2)
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep + 3, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        return lr

    lrfn(5)

    # if plot:  # Plot lr curve if plot is True
    #     plt.figure(figsize=(10, 5))
    #     plt.plot(np.arange(epochs), [lrfn(epoch) for epoch in np.arange(epochs)], marker='o')
    #     plt.xlabel('epoch'); plt.ylabel('lr')
    #     plt.title('LR Scheduler')
    #     plt.show()

    return keras.callbacks.LearningRateScheduler(lrfn, verbose=False) 

lr_cb = get_lr_callback(config.batch_size, mode=config.lr_mode, plot=True)

In [ ]:
nn1.name = "nn1"
print(nn1.name)
nn2.name = "nn2"
nn3.name = "nn3"

In [ ]:
import os
import pickle
import keras
from keras.callbacks import ModelCheckpoint

def train_neural_networks(networks, lr_cb):
    # make dir ../data/weights
    if not os.path.exists("../data/weights"):
        os.makedirs("../data/weights")

    # make history dir 
    if not os.path.exists("../data/history"):
        os.makedirs("../data/history")

    histories = []

    for network in networks:
        print(f"started training {network}")

        if os.path.exists(f"../data/weights/{network.name}.keras"):
            print(f"weights for {network.name} already exist, skipping training")
            continue

        ckpt_cb = keras.callbacks.ModelCheckpoint(
            f"../data/weights/{network.name}.keras",
            monitor="val_head_r2",
            save_best_only=True,
            save_weights_only=False,
            mode="max",
        )

        history = network.fit(
            train_ds_i,
            epochs=config.epochs,
            callbacks=[lr_cb, ckpt_cb],
            steps_per_epoch=len(train_df) // config.batch_size,
            validation_data=valid_ds_i,
            verbose=config.verbose,
        )

        histories.append((history.history, ckpt_cb))
        # pickle dump history.history 
        with open(f"../data/history/{network.name}.pkl", "wb") as f:
            pickle.dump(history.history, f)


In [ ]:
nn1_test_2 = Model(config, "efficientnetv2_b2_imagenet", True, "nn1_test_2")
train_neural_networks([nn1_test_2], lr_cb)

In [ ]:
nn1 = Model(config, "efficientnetv2_b2_imagenet", True, "nn1")
nn2 = Model(config, "resnet50_imagenet", True, "nn2")
nn3 = Model(config, "efficientnetv2_s_imagenet", True, "nn3")
# train_neural_networks([nn1, nn2, nn3], lr_cb)

In [ ]:
nn_no_train = Model(config, "efficientnetv2_b2_imagenet", False, "nn_no_train")
# train_neural_networks([nn_no_train], lr_cb)

In [ ]:
cos_lr_cb = get_lr_callback(config.batch_size, mode='cos', epochs=config.epochs, plot=True)
exp_lr_cb = get_lr_callback(config.batch_size, mode='exp', epochs=config.epochs, plot=True)

nn_cos_lr_cb = Model(config, "efficientnetv2_b2_imagenet", True, "nn_cos_lr_cb")
nn_exp_lr_cb = Model(config, "efficientnetv2_b2_imagenet", True, "nn_exp_lr_cb")

# train_neural_networks([nn_cos_lr_cb, nn_exp_lr_cb], cos_lr_cb)

### visualizations

Ok so we wanna plot the different r2 loss for the diff models with their respective backbones

In [ ]:
nn1.summary()

In [ ]:
# load history.history as ../data/functional_1.pkl
histories = []
for i in os.listdir("../data/history"):
    if not i.endswith(".pkl"):
        continue
    
    with open(f"../data/history/{i}", "rb") as f:
        print(i)
        histories.append((pickle.load(f), i))

fig, ax = plt.subplots(2, 6, figsize=(30, 10))

for i, (history, name) in enumerate(histories):
    ax[0, i].plot(history["head_r2"], label="head_r2")
    ax[0, i].plot(history["val_head_r2"], label="val_head_r2")
    ax[0, i].set_title(f"{name} head_r2")
    ax[0, i].set_xlabel("Epoch")
    ax[0, i].set_ylabel("R2")
    ax[0, i].legend()

    ax[1, i].plot(history["loss"], label="loss")
    ax[1, i].plot(history["val_loss"], label="val_loss")
    ax[1, i].set_title(f"{name} loss")
    ax[1, i].set_xlabel("Epoch")
    ax[1, i].set_ylabel("Loss")
    ax[1, i].legend()
    
fig.tight_layout()


In [ ]:
nn1 = keras.models.load_model("../data/weights/nn1.keras")
nn1.summary()

In [ ]:
DATA_PATH = "../data"

# we wanna load the models
nn1 = keras.models.load_model(f"../data/weights/nn1.keras")

# sample image
img_path = f'{DATA_PATH}/train_images/26375.jpeg'

In [ ]:
nn1.layers

We can check the intermediate activations here

In [ ]:
from keras.models import Model
import numpy as np
import cv2

# Load the model
nn1 = keras.models.load_model("../data/weights/nn1.keras")

# Load and preprocess the image
img_path = "../data/train_images/26375.jpeg"
img = cv2.imread(img_path)
img = cv2.resize(img, (224, 224))  
img = np.expand_dims(img, axis=0) 
img = img / 255.0 

# Create a new model that outputs the output of the dense_1 layer
dense_output_model = Model(inputs=nn1.input, outputs=nn1.get_layer('dense').output)
dense_1_output_model = Model(inputs=nn1.input, outputs=nn1.get_layer('dense_1').output)
dropout_output_model = Model(inputs=nn1.input, outputs=nn1.get_layer('dropout').output)

# Get the output of the dense_1 layer for your image
dense = dense_output_model.predict(img)
dense_1 = dense_1_output_model.predict(img)
dropout = dropout_output_model.predict(img)

fig, ax = plt.subplots(1, 3, figsize=(30, 10))
dense_df = pd.DataFrame(dense)
dense_1_df = pd.DataFrame(dense_1)
dropout_df = pd.DataFrame(dropout)

ax[0].plot(dense_df.T)
ax[0].set_title("Dense Layer")
ax[0].set_xlabel("Neuron")

ax[1].plot(dense_1_df.T)
ax[1].set_title("Dense_1 Layer")
ax[1].set_xlabel("Neuron")

ax[2].plot(dropout_df.T)
ax[2].set_title("Dropout Layer")
ax[2].set_xlabel("Neuron")

### evaluation + tuning

In [ ]:
# evaluting model and tuning hyperparameters

# training, validation acc 

# training, validation loss


## ensemble model

In [ ]:
%load_ext autoreload
%autoreload 2

keras.utils.set_random_seed(config.seed)
from combined_model import cModel
from model import Model
from config import config

(branch, img_input) = Model(config, "efficientnetv2_b2_imagenet", True, "img_model", True)

comb_model = cModel(config, FEATURE_COLS, (branch, img_input), "img_geo_1")

plot_model(comb_model, to_file='model.png', show_shapes=True, show_layer_names=True)

### training

In [ ]:
import os
import pickle
import keras
from keras.callbacks import ModelCheckpoint

def train_comb_neural_networks(networks, lr_cb):
    # make dir ../data/weights
    if not os.path.exists("../data/weights"):
        os.makedirs("../data/weights")

    # make history dir 
    if not os.path.exists("../data/history"):
        os.makedirs("../data/history")

    histories = []

    for network in networks:
        print(f"started training {network}")

        if os.path.exists(f"../data/weights/{network.name}.keras"):
            print(f"weights for {network.name} already exist, skipping training")
            continue

        ckpt_cb = keras.callbacks.ModelCheckpoint(
            f"../data/weights/{network.name}.keras",
            monitor="val_head_r2",
            save_best_only=True,
            save_weights_only=False,
            mode="max",
        )

        history = network.fit(
            train_ds_ig,
            epochs=config.epochs,
            callbacks=[lr_cb, ckpt_cb],
            steps_per_epoch=len(train_df) // config.batch_size,
            validation_data=valid_ds,
            verbose=config.verbose,
        )

        histories.append((history.history, ckpt_cb))
        # pickle dump history.history 
        with open(f"../data/history/{network.name}.pkl", "wb") as f:
            pickle.dump(history.history, f)


In [ ]:
train_comb_neural_networks([comb_model], lr_cb)

### evaluation + tuning

In [ ]:
# evaluating ensemble model and tuning hyperparameters

## ensemble vs image model 

### performance evaluation

### conclusion 

( you can use any notes here in the paper )